<a href="https://colab.research.google.com/github/DoItSon/playdata/blob/main/%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D/%EB%B0%B1%ED%99%94%EC%A0%90_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
DATA_PATH = "/content/drive/MyDrive/data/"
tr_train = pd.read_csv(f"{DATA_PATH}trans_train.csv")
target_train = pd.read_csv(f"{DATA_PATH}target_train.csv")

tr_train.shape , target_train.shape

((167465, 7), (2554, 2))

In [ ]:
tr_train["구매일자"] = pd.to_datetime(tr_train["구매일자"])
agg_dict = {
    '구매일자':[
        ("내점일수","nunique"),
        ("구매주기",lambda x : (x.max() - x.min()).days / x.nunique() ), # 최근 구매일과 최초 구매일의 차이를 구하고 내점일수로 나누면 구매주기라고 볼수있음
        ("주말방문비율", lambda x : (x.dt.weekday > 4).mean() ), # 4보다 크면 토,일 이니까!!
        ("주구매요일" , lambda x: x.dt.weekday.mode()[0] ), # 최빈값이 여러개 일수도 있어서..
        ("봄구매비율" , lambda x : (x.dt.month.isin([3,4,5])).mean() ),
        ("여름구매비율" , lambda x : (x.dt.month.isin([6,7,8])).mean() ),
        ("가을구매비율" , lambda x : (x.dt.month.isin([9,10,11])).mean() ),
        ("겨울구매비율" , lambda x : (x.dt.month.isin([12,1,2])).mean() ),
        ("일평균구매건수" , lambda x : x.count() / x.nunique() ), # 구매건수를 구매일자로 나누면 일평균 구매건수라고 볼수있음.
        ("거래개월수" , lambda x : x.dt.month.nunique() )
    ],
    '지점':[
         ("방문지점수","nunique"),
         ("주구매지점" , lambda x : x.mode()[0])
    ],
    '상품번호': [
             ('상품번호수', 'nunique'),
             ],
    '중분류': [
            ('중분류수', 'nunique'),
            ('주구매중분류', lambda x: x.mode()[0]),
            ],
    '대분류': [
            ('대분류수', 'nunique'),
            ('주구매대분류', lambda x: x.mode()[0]),
            ],
    '구매가격' : [
        ('총구매액','sum'),
        ('구매건수', 'count'),
        ('평균구매액', "mean"),
        ('최대구매액', 'max'),
        ('최소구매액',lambda x: x[x > 0].min() ) ,
        ('환불금액',lambda x: x[x < 0].sum() ) ,
        ('환불건수', lambda x: ( x < 0 ).sum() ),
        ('구매금액표준편차',lambda x: x[x>0].std() ),
    ],
}
tmp = tr_train.groupby('고객번호').agg(agg_dict)
tmp.columns = tmp.columns.droplevel()
tmp = tmp.reset_index()
target_train = target_train.merge(tmp,how="left",on = "고객번호")
target_train

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,대분류수,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,25,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,9,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,17,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,15,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,20,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,20,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000,2,5.931527e+05
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,20,농산물,54369820,121,4.493374e+05,7500000,1100.0,0,0,1.148466e+06
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,1,명품,1657400,3,5.524667e+05,1015000,39000.0,0,0,4.899894e+05
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,7,농산물,40742400,11,3.703855e+06,21780000,36000.0,0,0,6.610017e+06


In [ ]:
lst = ["농산물","가공식품","축산가공","수산품"]
agg_list = [
            ('식자재_구매수', lambda x: (x.isin(lst)).sum()), # count는 NaN데이터를 제외한 행의 수를 센다
            ]                                                   # sum은 True, False의 수를 센다.

tmp = tr_train.groupby('고객번호')["대분류"].agg(agg_list).reset_index()
tmp
target_train = pd.merge(target_train,tmp,how="left",on="고객번호")
target_train

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,주구매대분류,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,식자재_구매수
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,가공식품,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,11
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,가공식품,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,9
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,골프,81765810,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,9
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,명품,213705680,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,15
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,시티웨어,358336570,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,가공식품,47428060,371,1.278384e+05,9120000,1000.0,-1532000,2,5.931527e+05,243
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,농산물,54369820,121,4.493374e+05,7500000,1100.0,0,0,1.148466e+06,58
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,명품,1657400,3,5.524667e+05,1015000,39000.0,0,0,4.899894e+05,0
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,농산물,40742400,11,3.703855e+06,21780000,36000.0,0,0,6.610017e+06,5


In [ ]:
lst = ["명품","보석","모피/피혁","액세서리","준보석/시계"]
agg_list = [
            ('사치용품_구매수', lambda x: (x.isin(lst)).sum()), # count는 NaN 데이터를 제외한 행의 수를 센다
            ]
tmp = tr_train.groupby('고객번호')["대분류"].agg(agg_list).reset_index()
target_train = pd.merge(target_train,tmp,how="left",on="고객번호")
target_train

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,총구매액,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,식자재_구매수,사치용품_구매수
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,390166360,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,11,5
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,22447800,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,9,0
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,81765810,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,9,1
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,213705680,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,15,19
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,358336570,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,25,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,47428060,371,1.278384e+05,9120000,1000.0,-1532000,2,5.931527e+05,243,5
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,54369820,121,4.493374e+05,7500000,1100.0,0,0,1.148466e+06,58,0
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,1657400,3,5.524667e+05,1015000,39000.0,0,0,4.899894e+05,0,3
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,40742400,11,3.703855e+06,21780000,36000.0,0,0,6.610017e+06,5,0


In [ ]:
agg_list = [
    # (컬럼명, 집계방식)
    ("거래개월수X내점일수" , lambda x :x.nunique() * x.dt.month.nunique())
]

tmp = tr_train.groupby("고객번호")["구매일자"].agg(agg_list).reset_index()
target_train = target_train.merge(tmp,how="left",on = "고객번호")
target_train

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,구매건수,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,식자재_구매수,사치용품_구매수,거래개월수X내점일수
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,68,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,11,5,234
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,17,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,9,0,55
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,43,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,9,1,231
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,66,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,15,19,240
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,111,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,25,3,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,371,1.278384e+05,9120000,1000.0,-1532000,2,5.931527e+05,243,5,996
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,121,4.493374e+05,7500000,1100.0,0,0,1.148466e+06,58,0,341
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,3,5.524667e+05,1015000,39000.0,0,0,4.899894e+05,0,3,6
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,11,3.703855e+06,21780000,36000.0,0,0,6.610017e+06,5,0,42


In [ ]:
def do_apply(x):
    mask = x["구매일자"].dt.weekday < 5
    return (x.loc[mask, "구매가격"]<0).sum()

tmp = tr_train.groupby("고객번호").apply(do_apply)
tmp = tmp.reset_index()
tmp = tmp.rename(columns = {0:"평일환불수"})

target_train = target_train.merge(tmp,how="left",on = "고객번호")
target_train

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,평균구매액,최대구매액,최소구매액,환불금액,환불건수,구매금액표준편차,식자재_구매수,사치용품_구매수,거래개월수X내점일수,평일환불수
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,5.737741e+06,41760000,6570.0,-4700000,1,9.453819e+06,11,5,234,0
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,1.320459e+06,13375000,6900.0,0,0,3.179888e+06,9,0,55,0
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,1.901530e+06,14325000,1250.0,-6003000,1,3.738478e+06,9,1,231,1
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,3.237965e+06,24738000,1500.0,-12948000,3,5.883481e+06,15,19,240,2
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,3.228257e+06,38328000,1250.0,-30868000,3,6.959274e+06,25,3,252,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,49813,0,83,4.253012,0.167116,1,0.226415,0.339623,0.237197,0.196765,...,1.278384e+05,9120000,1000.0,-1532000,2,5.931527e+05,243,5,996,1
2550,49892,0,31,10.096774,0.363636,6,0.181818,0.289256,0.380165,0.148760,...,4.493374e+05,7500000,1100.0,0,0,1.148466e+06,58,0,341,0
2551,49925,0,3,18.333333,0.333333,3,0.000000,1.000000,0.000000,0.000000,...,5.524667e+05,1015000,39000.0,0,0,4.899894e+05,0,3,6,0
2552,49943,0,7,30.285714,0.545455,5,0.090909,0.545455,0.272727,0.090909,...,3.703855e+06,21780000,36000.0,0,0,6.610017e+06,5,0,42,0


In [ ]:
target_train.select_dtypes("object").nunique()

주구매지점      24
주구매중분류    169
주구매대분류     40
dtype: int64

In [ ]:
target_train.isnull().sum() # 결측치 확인!

고객번호            0
target          0
내점일수            0
구매주기            0
주말방문비율          0
주구매요일           0
봄구매비율           0
여름구매비율          0
가을구매비율          0
겨울구매비율          0
일평균구매건수         0
거래개월수           0
방문지점수           0
주구매지점           0
상품번호수           0
중분류수            0
주구매중분류          0
대분류수            0
주구매대분류          0
총구매액            0
구매건수            0
평균구매액           0
최대구매액           0
최소구매액           1
환불금액            0
환불건수            0
구매금액표준편차      233
식자재_구매수         0
사치용품_구매수        0
거래개월수X내점일수      0
평일환불수           0
dtype: int64

In [ ]:
target_train["최소구매액"] = target_train["최소구매액"].fillna(target_train["최소구매액"].min())
target_train["구매금액표준편차"] = target_train["구매금액표준편차"].fillna(target_train["구매금액표준편차"].mean())
target_train.isnull().sum().sum()

0

In [ ]:
target_train.select_dtypes("object").nunique()

주구매지점      24
주구매중분류    169
주구매대분류     40
dtype: int64

In [ ]:
cols = target_train.select_dtypes("object").columns
cols

Index(['주구매지점', '주구매중분류', '주구매대분류'], dtype='object')

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
enc.fit_transform(target_train[cols]).toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [ ]:
tmp = pd.DataFrame(
    enc.fit_transform(target_train[cols]).toarray(), # 넘파이 배열
    columns = enc.get_feature_names_out()
)
tmp.head()

,주구매지점_강남점,주구매지점_관악점,주구매지점_광주점,주구매지점_노원점,주구매지점_대구점,주구매지점_대전점,주구매지점_동래점,주구매지점_미아점,주구매지점_본 점,주구매지점_부산본점,...,주구매대분류_주방용품,주구매대분류_차/커피,주구매대분류_축산가공,주구매대분류_침구/수예,주구매대분류_캐주얼,주구매대분류_커리어,주구매대분류_통신/컴퓨터,주구매대분류_트래디셔널,주구매대분류_피혁잡화,주구매대분류_화장품
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
data = pd.concat([target_train,tmp],axis = 1).drop(columns=cols)
data.head()

,고객번호,target,내점일수,구매주기,주말방문비율,주구매요일,봄구매비율,여름구매비율,가을구매비율,겨울구매비율,...,주구매대분류_주방용품,주구매대분류_차/커피,주구매대분류_축산가공,주구매대분류_침구/수예,주구매대분류_캐주얼,주구매대분류_커리어,주구매대분류_통신/컴퓨터,주구매대분류_트래디셔널,주구매대분류_피혁잡화,주구매대분류_화장품
0,10004,1,26,11.423077,0.617647,5,0.161765,0.397059,0.250000,0.191176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10013,0,11,21.090909,0.117647,4,0.352941,0.235294,0.352941,0.058824,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10023,0,21,16.666667,0.348837,3,0.348837,0.162791,0.232558,0.255814,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10040,0,24,13.833333,0.121212,4,0.272727,0.151515,0.318182,0.257576,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10052,0,28,8.892857,0.333333,2,0.585586,0.243243,0.171171,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x_data = data.iloc[:,2:] # 내점일수부터 필요하기 때문에 슬라이싱
y_data = data["target"]

x_data.shape, y_data.shape

((2554, 259), (2554,))

In [ ]:
scaler = MinMaxScaler()

x_data = scaler.fit_transform(x_data)
x_data

array([[0.08802817, 0.06881372, 0.61764706, ..., 0.        , 0.        ,
        0.        ],
       [0.03521127, 0.12705367, 0.11764706, ..., 0.        , 0.        ,
        0.        ],
       [0.07042254, 0.10040161, 0.34883721, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.00704225, 0.11044177, 0.33333333, ..., 0.        , 0.        ,
        0.        ],
       [0.02112676, 0.18244406, 0.54545455, ..., 0.        , 0.        ,
        0.        ],
       [0.01408451, 0.21445783, 0.04761905, ..., 0.        , 0.        ,
        0.        ]])

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x_data, y_data, test_size=0.2, random_state=SEED)

x_train.shape, x_valid.shape, y_train.shape, y_valid.shape

((2043, 259), (511, 259), (2043,), (511,))

In [ ]:
SEED = 42
model = LogisticRegression(random_state=SEED)

model.fit(x_train,y_train)

LogisticRegression(random_state=42)

In [ ]:
pred = model.predict(x_valid)
accuracy_score(y_valid,pred)

0.6888454011741683